In [1]:
import os 
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-GPU-bb1ccb6e-2bc9-c7a1-b25d-3eef9033e192/6/0"

In [2]:
"""
This requires torchtext
"""
import torch
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from pytorch_ood.dataset.txt import NewsGroup20, Reuters52, WMT16Sentences, Multi30k
from pytorch_ood.model.gruclf import GRUClassifier
from pytorch_ood.utils import ToUnknown, OODMetrics
from pytorch_ood.detector import MaxSoftmax, EnergyBased

torch.manual_seed(123)

n_epochs = 10
lr = 0.001

In [3]:
# download datasets
train_dataset = NewsGroup20("data/", train=True, download=True)
test_dataset = NewsGroup20("data/", train=False, download=True)

In [4]:
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_dataset)) # , max_tokens=10000# , specials=["<unk>"]
# vocab.set_default_index(0)

11293lines [00:00, 15811.16lines/s]


In [5]:
from torch.utils.data import Dataset
import torch 

class MyDataset():
    def __init__(self, n_dict=10000, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.n_dict = n_dict
    
    def __len__(self):
        return 80000
    
    def __getitem__(self, key):
        length = 20 # torch.randint(low=1, high=50, size=(1,))
        x = torch.randint(low=0, high=self.n_dict, size=(length,))
        y = 0
        
        if self.transform:
            x = self.transform(x)
            
        if self.target_transform:
            y = self.target_transform(y)
            
        return x, y

In [6]:
def prep(x):
    return torch.tensor([vocab[v] for v in tokenizer(x)], dtype=torch.int64)

train_in_dataset = NewsGroup20("data/", train=True, transform=prep)
test_dataset = NewsGroup20("data/", train=False, transform=prep)

train_out_dataset = MyDataset(target_transform=ToUnknown())  # transform=prep, 


In [7]:
# add padding, etc.
def collate_batch(batch):
    texts = [i[0] for i in batch]
    labels = torch.tensor([i[1] for i in batch],  dtype=torch.int64)
    t_lengths = torch.tensor([len(t) for t in texts])
    max_t_length = torch.max(t_lengths)

    padded = []
    for text in texts:
        t = torch.cat([torch.zeros(max_t_length-len(text), dtype=torch.long), text])
        padded.append(t)
    return torch.stack(padded,dim=0), labels

train_loader = DataLoader(train_in_dataset + train_out_dataset, batch_size=32, shuffle=True, 
                          collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=True, collate_fn=collate_batch)


In [8]:
model = GRUClassifier(num_classes=20, n_vocab=len(vocab))
model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)

In [9]:
import torch.nn.functional as F
from pytorch_ood.loss import OutlierExposureLoss

model.train()
model.cuda()

criterion = OutlierExposureLoss(alpha=0.5)


for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    loss_ema = 0
    correct = 0
    total = 0

    model.train()
    for n, batch in enumerate(train_loader):
        inputs, labels = batch

        inputs = inputs.cuda()
        labels = labels.cuda()
        logits = model(inputs)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_ema = loss_ema * 0.9 + loss.data.cpu().numpy() * 0.1

        pred = logits.max(dim=1).indices
        correct += pred.eq(labels).sum().data.cpu().numpy()
        total += pred.shape[0]

        if n % 10 == 0:
            print(f"Loss: {loss_ema.item():.2f} Accuracy {correct/total:.2%}")

    with torch.no_grad():
        model.eval()
        correct = 0
        total = 0
        for n, batch in enumerate(test_loader):
            inputs, labels = batch

            inputs = inputs.cuda()
            labels = labels.cuda()
            logits = model(inputs)
            pred = logits.max(dim=1).indices
            correct += pred.eq(labels).sum().data.cpu().numpy()
            total += pred.shape[0]

        print(f"Test Accuracy: {correct/total:.2%}")

Epoch 0
Loss: 0.16 Accuracy 0.00%
Loss: 1.16 Accuracy 0.57%
Loss: 1.49 Accuracy 0.89%
Loss: 1.61 Accuracy 0.60%
Loss: 1.68 Accuracy 0.61%
Loss: 1.69 Accuracy 0.86%
Loss: 1.71 Accuracy 0.92%
Loss: 1.69 Accuracy 0.84%
Loss: 1.69 Accuracy 0.93%
Loss: 1.68 Accuracy 0.93%
Loss: 1.69 Accuracy 0.93%
Loss: 1.68 Accuracy 0.96%
Loss: 1.69 Accuracy 0.98%
Loss: 1.67 Accuracy 0.95%
Loss: 1.65 Accuracy 0.93%
Loss: 1.69 Accuracy 1.03%
Loss: 1.70 Accuracy 0.99%
Loss: 1.67 Accuracy 0.95%
Loss: 1.65 Accuracy 0.92%
Loss: 1.68 Accuracy 0.97%
Loss: 1.67 Accuracy 1.03%
Loss: 1.65 Accuracy 1.05%
Loss: 1.65 Accuracy 1.06%
Loss: 1.68 Accuracy 1.12%
Loss: 1.70 Accuracy 1.15%
Loss: 1.67 Accuracy 1.15%
Loss: 1.68 Accuracy 1.13%
Loss: 1.66 Accuracy 1.14%
Loss: 1.68 Accuracy 1.15%
Loss: 1.68 Accuracy 1.16%
Loss: 1.67 Accuracy 1.16%
Loss: 1.66 Accuracy 1.24%
Loss: 1.67 Accuracy 1.26%
Loss: 1.65 Accuracy 1.26%
Loss: 1.69 Accuracy 1.26%
Loss: 1.68 Accuracy 1.28%
Loss: 1.68 Accuracy 1.26%
Loss: 1.69 Accuracy 1.27%
Loss

Loss: 1.54 Accuracy 5.34%
Loss: 1.52 Accuracy 5.39%
Loss: 1.52 Accuracy 5.39%
Loss: 1.53 Accuracy 5.38%
Loss: 1.56 Accuracy 5.48%
Loss: 1.55 Accuracy 5.55%
Loss: 1.53 Accuracy 5.57%
Loss: 1.51 Accuracy 5.60%
Loss: 1.51 Accuracy 5.64%
Loss: 1.49 Accuracy 5.74%
Loss: 1.51 Accuracy 5.71%
Loss: 1.50 Accuracy 5.68%
Loss: 1.52 Accuracy 5.73%
Loss: 1.54 Accuracy 5.73%
Loss: 1.52 Accuracy 5.74%
Loss: 1.51 Accuracy 5.76%
Loss: 1.49 Accuracy 5.81%
Loss: 1.50 Accuracy 5.80%
Loss: 1.50 Accuracy 5.80%
Loss: 1.52 Accuracy 5.77%
Loss: 1.55 Accuracy 5.78%
Loss: 1.54 Accuracy 5.81%
Loss: 1.51 Accuracy 5.83%
Loss: 1.53 Accuracy 5.81%
Loss: 1.52 Accuracy 5.81%
Loss: 1.53 Accuracy 5.80%
Loss: 1.52 Accuracy 5.78%
Loss: 1.51 Accuracy 5.78%
Loss: 1.54 Accuracy 5.75%
Loss: 1.55 Accuracy 5.72%
Loss: 1.51 Accuracy 5.72%
Loss: 1.50 Accuracy 5.72%
Loss: 1.51 Accuracy 5.69%
Loss: 1.49 Accuracy 5.73%
Loss: 1.53 Accuracy 5.75%
Loss: 1.54 Accuracy 5.74%
Loss: 1.54 Accuracy 5.75%
Loss: 1.52 Accuracy 5.75%
Loss: 1.52 A

Loss: 1.43 Accuracy 9.22%
Loss: 1.41 Accuracy 9.24%
Loss: 1.42 Accuracy 9.24%
Loss: 1.47 Accuracy 9.20%
Loss: 1.42 Accuracy 9.21%
Loss: 1.42 Accuracy 9.20%
Loss: 1.41 Accuracy 9.21%
Loss: 1.41 Accuracy 9.22%
Loss: 1.41 Accuracy 9.22%
Loss: 1.43 Accuracy 9.25%
Loss: 1.41 Accuracy 9.27%
Loss: 1.44 Accuracy 9.25%
Loss: 1.43 Accuracy 9.24%
Loss: 1.43 Accuracy 9.24%
Loss: 1.44 Accuracy 9.25%
Loss: 1.42 Accuracy 9.27%
Loss: 1.42 Accuracy 9.27%
Loss: 1.43 Accuracy 9.26%
Loss: 1.41 Accuracy 9.26%
Loss: 1.40 Accuracy 9.27%
Loss: 1.41 Accuracy 9.28%
Loss: 1.42 Accuracy 9.26%
Loss: 1.42 Accuracy 9.26%
Loss: 1.43 Accuracy 9.24%
Loss: 1.41 Accuracy 9.26%
Loss: 1.42 Accuracy 9.26%
Loss: 1.38 Accuracy 9.31%
Loss: 1.39 Accuracy 9.33%
Loss: 1.39 Accuracy 9.34%
Loss: 1.42 Accuracy 9.32%
Loss: 1.44 Accuracy 9.31%
Loss: 1.43 Accuracy 9.32%
Loss: 1.43 Accuracy 9.29%
Loss: 1.45 Accuracy 9.29%
Loss: 1.45 Accuracy 9.28%
Loss: 1.42 Accuracy 9.28%
Loss: 1.44 Accuracy 9.29%
Loss: 1.42 Accuracy 9.29%
Loss: 1.43 A

Loss: 1.36 Accuracy 10.76%
Loss: 1.36 Accuracy 10.77%
Loss: 1.37 Accuracy 10.78%
Loss: 1.38 Accuracy 10.79%
Loss: 1.35 Accuracy 10.80%
Loss: 1.37 Accuracy 10.78%
Loss: 1.35 Accuracy 10.81%
Loss: 1.34 Accuracy 10.83%
Loss: 1.39 Accuracy 10.79%
Loss: 1.35 Accuracy 10.82%
Loss: 1.36 Accuracy 10.82%
Loss: 1.38 Accuracy 10.80%
Loss: 1.37 Accuracy 10.80%
Loss: 1.37 Accuracy 10.80%
Loss: 1.38 Accuracy 10.80%
Loss: 1.38 Accuracy 10.79%
Loss: 1.36 Accuracy 10.80%
Loss: 1.38 Accuracy 10.79%
Loss: 1.40 Accuracy 10.76%
Loss: 1.36 Accuracy 10.77%
Loss: 1.36 Accuracy 10.78%
Loss: 1.38 Accuracy 10.76%
Loss: 1.36 Accuracy 10.77%
Loss: 1.38 Accuracy 10.77%
Loss: 1.37 Accuracy 10.78%
Loss: 1.34 Accuracy 10.82%
Loss: 1.36 Accuracy 10.81%
Loss: 1.38 Accuracy 10.81%
Loss: 1.35 Accuracy 10.82%
Loss: 1.30 Accuracy 10.86%
Loss: 1.34 Accuracy 10.85%
Loss: 1.37 Accuracy 10.84%
Loss: 1.35 Accuracy 10.85%
Loss: 1.37 Accuracy 10.84%
Loss: 1.38 Accuracy 10.84%
Loss: 1.38 Accuracy 10.83%
Loss: 1.36 Accuracy 10.85%
L

Loss: 1.30 Accuracy 11.78%
Loss: 1.33 Accuracy 11.78%
Loss: 1.31 Accuracy 11.81%
Loss: 1.34 Accuracy 11.80%
Loss: 1.34 Accuracy 11.79%
Loss: 1.34 Accuracy 11.79%
Loss: 1.38 Accuracy 11.76%
Loss: 1.36 Accuracy 11.75%
Loss: 1.37 Accuracy 11.74%
Loss: 1.38 Accuracy 11.71%
Loss: 1.34 Accuracy 11.72%
Loss: 1.35 Accuracy 11.70%
Loss: 1.31 Accuracy 11.73%
Loss: 1.34 Accuracy 11.72%
Loss: 1.40 Accuracy 11.68%
Loss: 1.38 Accuracy 11.67%
Loss: 1.38 Accuracy 11.64%
Loss: 1.35 Accuracy 11.65%
Loss: 1.39 Accuracy 11.62%
Loss: 1.34 Accuracy 11.64%
Loss: 1.35 Accuracy 11.63%
Loss: 1.37 Accuracy 11.60%
Loss: 1.35 Accuracy 11.61%
Loss: 1.31 Accuracy 11.63%
Loss: 1.35 Accuracy 11.61%
Loss: 1.31 Accuracy 11.64%
Loss: 1.32 Accuracy 11.64%
Loss: 1.29 Accuracy 11.68%
Loss: 1.33 Accuracy 11.68%
Loss: 1.32 Accuracy 11.70%
Loss: 1.34 Accuracy 11.68%
Loss: 1.34 Accuracy 11.68%
Loss: 1.34 Accuracy 11.68%
Loss: 1.34 Accuracy 11.68%
Loss: 1.34 Accuracy 11.68%
Loss: 1.34 Accuracy 11.68%
Loss: 1.31 Accuracy 11.71%
L

Loss: 1.29 Accuracy 12.02%
Loss: 1.31 Accuracy 12.03%
Loss: 1.32 Accuracy 12.02%
Loss: 1.32 Accuracy 12.04%
Loss: 1.32 Accuracy 12.03%
Loss: 1.37 Accuracy 12.00%
Loss: 1.33 Accuracy 12.01%
Loss: 1.34 Accuracy 12.00%
Loss: 1.33 Accuracy 11.99%
Loss: 1.36 Accuracy 11.99%
Loss: 1.35 Accuracy 11.98%
Loss: 1.31 Accuracy 12.00%
Loss: 1.33 Accuracy 11.99%
Loss: 1.33 Accuracy 11.98%
Loss: 1.33 Accuracy 11.99%
Loss: 1.30 Accuracy 12.01%
Loss: 1.32 Accuracy 12.01%
Loss: 1.36 Accuracy 11.99%
Loss: 1.34 Accuracy 11.98%
Loss: 1.33 Accuracy 11.98%
Loss: 1.31 Accuracy 11.99%
Loss: 1.32 Accuracy 12.00%
Loss: 1.33 Accuracy 12.00%
Loss: 1.36 Accuracy 11.99%
Loss: 1.35 Accuracy 11.99%
Loss: 1.32 Accuracy 12.00%
Loss: 1.32 Accuracy 12.01%
Loss: 1.36 Accuracy 11.99%
Loss: 1.34 Accuracy 11.99%
Loss: 1.36 Accuracy 11.96%
Loss: 1.31 Accuracy 11.98%
Loss: 1.36 Accuracy 11.96%
Loss: 1.34 Accuracy 11.96%
Loss: 1.36 Accuracy 11.94%
Loss: 1.30 Accuracy 11.96%
Loss: 1.31 Accuracy 11.97%
Loss: 1.33 Accuracy 11.97%
L

Loss: 1.29 Accuracy 12.11%
Loss: 1.33 Accuracy 12.10%
Loss: 1.32 Accuracy 12.11%
Loss: 1.34 Accuracy 12.10%
Loss: 1.33 Accuracy 12.10%
Loss: 1.34 Accuracy 12.09%
Loss: 1.35 Accuracy 12.08%
Loss: 1.39 Accuracy 12.05%
Loss: 1.34 Accuracy 12.04%
Loss: 1.32 Accuracy 12.05%
Loss: 1.34 Accuracy 12.04%
Loss: 1.36 Accuracy 12.03%
Loss: 1.32 Accuracy 12.03%
Loss: 1.33 Accuracy 12.03%
Loss: 1.31 Accuracy 12.04%
Loss: 1.35 Accuracy 12.02%
Loss: 1.34 Accuracy 12.01%
Loss: 1.33 Accuracy 12.01%
Loss: 1.31 Accuracy 12.02%
Loss: 1.34 Accuracy 12.01%
Loss: 1.35 Accuracy 12.01%
Loss: 1.31 Accuracy 12.03%
Loss: 1.32 Accuracy 12.03%
Loss: 1.32 Accuracy 12.04%
Loss: 1.32 Accuracy 12.04%
Loss: 1.31 Accuracy 12.05%
Loss: 1.31 Accuracy 12.06%
Loss: 1.29 Accuracy 12.08%
Loss: 1.32 Accuracy 12.08%
Loss: 1.33 Accuracy 12.07%
Loss: 1.32 Accuracy 12.08%
Loss: 1.37 Accuracy 12.06%
Loss: 1.35 Accuracy 12.05%
Loss: 1.34 Accuracy 12.06%
Loss: 1.32 Accuracy 12.06%
Loss: 1.34 Accuracy 12.05%
Loss: 1.34 Accuracy 12.05%
L

Loss: 1.31 Accuracy 12.20%
Loss: 1.32 Accuracy 12.19%
Loss: 1.32 Accuracy 12.20%
Loss: 1.30 Accuracy 12.21%
Loss: 1.31 Accuracy 12.21%
Loss: 1.34 Accuracy 12.19%
Loss: 1.36 Accuracy 12.19%
Loss: 1.34 Accuracy 12.19%
Loss: 1.35 Accuracy 12.17%
Loss: 1.36 Accuracy 12.17%
Loss: 1.34 Accuracy 12.17%
Loss: 1.33 Accuracy 12.17%
Loss: 1.32 Accuracy 12.17%
Loss: 1.30 Accuracy 12.18%
Loss: 1.33 Accuracy 12.17%
Loss: 1.30 Accuracy 12.18%
Loss: 1.31 Accuracy 12.18%
Loss: 1.28 Accuracy 12.20%
Loss: 1.30 Accuracy 12.20%
Loss: 1.30 Accuracy 12.21%
Loss: 1.31 Accuracy 12.21%
Loss: 1.32 Accuracy 12.21%
Loss: 1.33 Accuracy 12.21%
Loss: 1.34 Accuracy 12.20%
Loss: 1.32 Accuracy 12.20%
Loss: 1.34 Accuracy 12.19%
Loss: 1.33 Accuracy 12.19%
Loss: 1.36 Accuracy 12.18%
Loss: 1.36 Accuracy 12.16%
Loss: 1.34 Accuracy 12.16%
Loss: 1.32 Accuracy 12.16%
Loss: 1.30 Accuracy 12.17%
Loss: 1.33 Accuracy 12.16%
Loss: 1.35 Accuracy 12.15%
Loss: 1.35 Accuracy 12.14%
Loss: 1.33 Accuracy 12.15%
Loss: 1.31 Accuracy 12.15%
L

Loss: 1.32 Accuracy 12.23%
Loss: 1.35 Accuracy 12.22%
Loss: 1.32 Accuracy 12.23%
Loss: 1.35 Accuracy 12.21%
Loss: 1.32 Accuracy 12.22%
Loss: 1.32 Accuracy 12.22%
Loss: 1.28 Accuracy 12.24%
Loss: 1.31 Accuracy 12.24%
Loss: 1.33 Accuracy 12.23%
Loss: 1.33 Accuracy 12.24%
Loss: 1.36 Accuracy 12.22%
Loss: 1.32 Accuracy 12.23%
Loss: 1.32 Accuracy 12.23%
Loss: 1.31 Accuracy 12.23%
Loss: 1.37 Accuracy 12.22%
Loss: 1.32 Accuracy 12.22%
Loss: 1.33 Accuracy 12.22%
Loss: 1.34 Accuracy 12.21%
Loss: 1.32 Accuracy 12.21%
Loss: 1.32 Accuracy 12.22%
Loss: 1.32 Accuracy 12.22%
Loss: 1.33 Accuracy 12.22%
Loss: 1.36 Accuracy 12.20%
Loss: 1.32 Accuracy 12.21%
Loss: 1.27 Accuracy 12.24%
Loss: 1.30 Accuracy 12.25%
Loss: 1.32 Accuracy 12.25%
Loss: 1.34 Accuracy 12.24%
Loss: 1.33 Accuracy 12.24%
Loss: 1.32 Accuracy 12.24%
Loss: 1.29 Accuracy 12.25%
Loss: 1.31 Accuracy 12.25%
Loss: 1.32 Accuracy 12.25%
Loss: 1.31 Accuracy 12.25%
Loss: 1.32 Accuracy 12.25%
Loss: 1.33 Accuracy 12.24%
Loss: 1.31 Accuracy 12.25%
L

Loss: 1.33 Accuracy 12.29%
Loss: 1.33 Accuracy 12.30%
Loss: 1.31 Accuracy 12.31%
Loss: 1.35 Accuracy 12.30%
Loss: 1.33 Accuracy 12.31%
Loss: 1.33 Accuracy 12.30%
Loss: 1.32 Accuracy 12.30%
Loss: 1.34 Accuracy 12.29%
Loss: 1.35 Accuracy 12.29%
Loss: 1.30 Accuracy 12.31%
Loss: 1.30 Accuracy 12.31%
Loss: 1.29 Accuracy 12.33%
Loss: 1.32 Accuracy 12.32%
Loss: 1.30 Accuracy 12.33%
Loss: 1.31 Accuracy 12.33%
Loss: 1.34 Accuracy 12.33%
Loss: 1.34 Accuracy 12.32%
Loss: 1.35 Accuracy 12.31%
Loss: 1.35 Accuracy 12.30%
Loss: 1.32 Accuracy 12.30%
Loss: 1.31 Accuracy 12.31%
Loss: 1.34 Accuracy 12.30%
Loss: 1.33 Accuracy 12.31%
Loss: 1.29 Accuracy 12.32%
Loss: 1.33 Accuracy 12.31%
Loss: 1.35 Accuracy 12.30%
Loss: 1.32 Accuracy 12.30%
Loss: 1.29 Accuracy 12.32%
Loss: 1.34 Accuracy 12.31%
Loss: 1.33 Accuracy 12.30%
Loss: 1.34 Accuracy 12.29%
Loss: 1.38 Accuracy 12.27%
Loss: 1.35 Accuracy 12.26%
Loss: 1.33 Accuracy 12.26%
Loss: 1.31 Accuracy 12.27%
Loss: 1.34 Accuracy 12.26%
Loss: 1.30 Accuracy 12.27%
L

In [10]:
def test(model, dataset, dataset_name):
    test_loader = DataLoader(dataset, batch_size=256, shuffle=True, collate_fn=collate_batch)
    metrics = OODMetrics()
    metrics_energy = OODMetrics()
    softmax = MaxSoftmax(model)
    energy = EnergyBased(model)
    model.eval()

    with torch.no_grad():
        for n, batch in enumerate(test_loader):
            inputs, labels = batch

            inputs = inputs.cuda()
            labels = labels.cuda()
            metrics.update(softmax(inputs), labels)
            metrics_energy.update(energy(inputs), labels)

    d1 = metrics.compute()
    d1.update({"Method": "Softmax", "Dataset": dataset_name})
    
    d2 = metrics_energy.compute()
    d2.update({"Method": "Energy", "Dataset": dataset_name})
    return [d1, d2]

res = []

ood_dataset = Reuters52("data/", train=False, download=True, transform=prep, target_transform=ToUnknown())
res+= test(model, test_dataset + ood_dataset, dataset_name="Reuters52")

ood_dataset = Multi30k("data/", train=False, download=True, transform=prep, target_transform=ToUnknown())
res+= test(model, test_dataset + ood_dataset, dataset_name="Multi30k")

ood_dataset = WMT16Sentences("data/", download=True, transform=prep, target_transform=ToUnknown())
res+= test(model, test_dataset + ood_dataset, dataset_name="WMT16Sentences")

/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: `pos_label` automatically set 1.
  warnings.warn(*args, **kwargs)
/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/functional/classification/precision_recall_curve.py:156: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().

In [11]:
import pandas as pd 
df = pd.DataFrame(res)
print((df.groupby("Method").mean() * 100).to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrr}
\toprule
{} &  AUROC &  AUPR-IN &  AUPR-OUT &  ACC95TPR &  FPR95TPR \\
Method  &        &          &           &           &           \\
\midrule
Energy  &  93.93 &    78.20 &     97.02 &     84.93 &     18.94 \\
Softmax &  93.22 &    77.65 &     96.42 &     82.06 &     23.11 \\
\bottomrule
\end{tabular}

